In [ ]:
import re
import tempfile
from pathlib import Path
from pyswmm.simulation import Simulation


# def scale_timeseries_in_inp(inp_path, scale_factor, ts_prefix="Q_", output_path=None):
#     """Return a copy of *inp_path* with every TIMESERIES that starts with ts_prefix
#     multiplied by *scale_factor*."""
#     inp_path = Path(inp_path)
#     lines    = inp_path.read_text().splitlines()

#     in_ts_block = False
#     new_lines   = []
#     for line in lines:
#         stripped = line.strip()

#         if stripped.upper().startswith("[TIMESERIES]"):
#             in_ts_block = True
#             new_lines.append(line)
#             continue

#         if in_ts_block and stripped.startswith("["):
#             in_ts_block = False             # left the section

#         if in_ts_block and stripped and not stripped.startswith(";"):
#             parts = re.split(r"\s+", stripped)
#             if len(parts) >= 3 and parts[0].startswith(ts_prefix):
#                 try:
#                     val       = float(parts[2])
#                     parts[2]  = f"{val * scale_factor:.4f}"
#                     line      = "    ".join(parts)
#                 except ValueError:
#                     pass                     # leave unchanged if parsing fails
#         new_lines.append(line)

#     out_path = Path(output_path or tempfile.mktemp(suffix=".inp"))
#     out_path.write_text("\n".join(new_lines))
#     return out_path

def set_inflow_scale_factor(
        inp_path: Path,
        ts_name: str,
        scale_factor: float,
        ts_prefix: str = "Q_",
        output_path: Path | None = None
) -> Path:
    """
    Return a copy of *inp_path* in which the INFLOWS record whose
    TimeSeriesID == *ts_name* (and starts with *ts_prefix*) has its
    Scale Factor field set to *scale_factor*.

    • Only that one inflow line is edited; all others stay as they are.
    • If the INFLOWS line looks like
            JUNC‑3   FLOW   Q_3_T5   1.0   0
      we replace the '1.0' with the new factor.
    """

    lines = Path(inp_path).read_text().splitlines()
    new_lines = []

    in_inflows = False
    for line in lines:
        stripped = line.strip()

        # Detect start/end of [INFLOWS] -------------------------------------
        if stripped.upper().startswith("[INFLOWS]"):
            in_inflows = True
            new_lines.append(line)
            continue
        if in_inflows and stripped.startswith("[") and not stripped.upper().startswith("[INFLOWS]"):
            in_inflows = False                    # left the section

        # Inside [INFLOWS] – change the right row ---------------------------
        if in_inflows and stripped and not stripped.startswith(";"):
            parts = re.split(r"\s+", stripped)

            # Two common layouts:
            #   1) Node Constituent TimeSeries Scale Baseline Pattern
            #   2) Node TimeSeries Scale  (Constituent omitted; not standard)
            #
            # We'll look for the timeseries column first.
            try:
                if ts_name == parts[2]:           # standard layout
                    # parts[3] = scale factor
                    parts[3] = f"{scale_factor:.4f}"
                    line = "    ".join(parts)
                elif ts_name == parts[1]:         # non‑standard
                    parts[2] = f"{scale_factor:.4f}"
                    line = "    ".join(parts)
            except IndexError:
                pass

        new_lines.append(line)

    out_path = Path(output_path or tempfile.mktemp(suffix=".inp"))
    out_path.write_text("\n".join(new_lines))
    return out_path

# def sum_flood_volume_ml(rpt_path):
#     """Parses a SWMM report and sums all values in the Total Flooding Volume column (ML)."""
#     rpt_path = Path(rpt_path)
#     total = 0.0
#     in_flood_section = False

#     for line in rpt_path.read_text().splitlines():
#         if "Node Flooding Summary" in line:
#             in_flood_section = True
#             continue
#         if in_flood_section:
#             if line.strip().startswith("Node"):
#                 continue
#             if not line.strip() or line.strip().startswith("==="):
#                 break
#             parts = line.split()
#             try:
#                 vol_ml = float(parts[5])  # second-to-last column
#                 total += vol_ml
#             except (IndexError, ValueError):
#                 continue
#     return total

def parse_total_flood_volume(rpt_file):
    with open(rpt_file, 'r', encoding='latin-1') as f:
        lines = f.readlines()

    start, end = None, None
    for i, line in enumerate(lines):
        if 'Node Flooding Summary' in line:
            start = i + 7
        elif start and ('Outfall Loading Summary' in line or 'END OF REPORT' in line):
            end = i
            break

    total_volume = 0.0
    for line in lines[start:end]:
        if not line.strip():
            continue
        parts = re.split(r'\s+', line.strip())
        if len(parts) < 6:
            continue
        try:
            volume = float(parts[5])  # Volume is in million liters
            total_volume += volume
        except ValueError:
            continue

    return total_volume


def run_swmm_simulation(inp_path):
    """Runs SWMM and returns total flood volume (in ML) from the report."""
    inp_path = Path(inp_path)
    rpt_path = inp_path.with_suffix(".rpt")

    with Simulation(str(inp_path)) as sim:
        sim.execute()  # required for PySWMM 2.x

    return parse_total_flood_volume(rpt_path)


In [15]:
import re
import tempfile
from pathlib import Path
from pyswmm.simulation import Simulation


# ──────────────────────────────────────────────────────────────────────────
# 1.  EDIT THE SCALE‑FACTOR IN  [INFLOWS]
# ──────────────────────────────────────────────────────────────────────────

def multiply_inflow_mfactor_no_baseline(
        inp_path: Path | str,
        ts_name: str,
        scale_factor: float,
        output_path: Path | None = None,
) -> Path:
    """Return a copy of *inp_path* with the Mfactor (last numeric token after TimeSeriesID)
       multiplied by *scale_factor*. Baseline is removed (i.e., not preserved or added)."""
    
    lines = Path(inp_path).read_text().splitlines()
    result = []
    in_inflows = False

    for line in lines:
        stripped = line.strip()

        if stripped.upper().startswith("[INFLOWS]"):
            in_inflows = True
            result.append(line)
            continue
        if in_inflows and stripped.startswith("[") and not stripped.upper().startswith("[INFLOWS]"):
            in_inflows = False

        if in_inflows and ts_name in stripped and not stripped.startswith(";"):
            parts = re.split(r"\s+", stripped)
            if len(parts) >= 4 and parts[2] == ts_name:
                # find all numeric fields after TimeSeriesID
                numeric_indices = []
                for i in range(3, len(parts)):
                    try:
                        float(parts[i])
                        numeric_indices.append(i)
                    except ValueError:
                        continue
                if numeric_indices:
                    mf_index = numeric_indices[-1]  # last numeric = Mfactor
                    mf = float(parts[mf_index])
                    parts[mf_index] = f"{mf * scale_factor:.4f}"
                    # keep only up to the modified Mfactor
                    parts = parts[:mf_index + 1]

                line = "    ".join(parts)
        result.append(line)

    out_path = Path(output_path or tempfile.mktemp(suffix=".inp"))
    out_path.write_text("\n".join(result))
    return out_path






# ──────────────────────────────────────────────────────────────────────────
# 2.  SUM FLOOD VOLUMES (ML) FROM THE REPORT
# ──────────────────────────────────────────────────────────────────────────
def parse_total_flood_volume(rpt_file):
    with open(rpt_file, "r", encoding="latin-1") as f:
        lines = f.readlines()

    start, end = None, None
    for i, line in enumerate(lines):
        if "Node Flooding Summary" in line:
            start = i + 7                 # header offset OK
        elif start and ("Outfall Loading Summary" in line or "END OF REPORT" in line):
            end = i
            break

    total_volume = 0.0
    for line in lines[start:end]:
        if not line.strip():
            continue
        parts = re.split(r"\s+", line.strip())
        try:
            total_volume += float(parts[-2])   # second‑to‑last = ML
        except (ValueError, IndexError):
            continue

    return total_volume



# ──────────────────────────────────────────────────────────────────────────
# 3.  RUN A MODEL AND RETURN TOTAL FLOOD VOLUME (ML)
# ──────────────────────────────────────────────────────────────────────────
def run_swmm_total_flood_ml(inp_path: Path) -> float:
    """
    Execute SWMM (PySWMM ≥ 2.0) and return the summed flood volume in ML.
    """
    rpt_path = Path(inp_path).with_suffix(".rpt")
    with Simulation(str(inp_path)) as sim:
        sim.execute()                  # PySWMM 2.x call
    return parse_total_flood_volume(rpt_path)


In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_2_xsec_bar2.inp")

all_inflow_ts = [
    "Q_2_T_2", "Q_3_T_2", "Q_4_T_2", "Q_5_T_2", "Q_6_T_2", "Q_7_T_2", "Q_8_T_2", "Q_14_T_2", "Q_17_T_2" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T2 = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T2  = run_swmm_total_flood_ml(mod_inp)
        results_T2.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T2}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T2:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T2 = pd.DataFrame(results_T2)

# Extract numeric gauge number for sensible sorting
df_T2['gauge_num'] = df_T2['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T2 = df_T2.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T2['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T2.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T2 = df_T2.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T2 = vol_matrix_T2.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T2 = 1390
percent_decrease_matrix_T2 = (baseline_T2 - vol_matrix_T2) / baseline_T2 * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T2, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 2 Flood Volume (ML) for scaled inflows, baseline: {baseline_T2} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T2, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=2 Percentage Decrease in Flood Volume, baseline {baseline_T2} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)



# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_2_wflowforcing.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_5_xsec_bar2.inp")

all_inflow_ts = [
    "Q_2_T_5", "Q_3_T_5", "Q_4_T_5", "Q_5_T_5", "Q_6_T_5", "Q_7_T_5", "Q_8_T_5", "Q_14_T_5", "Q_17_T_5" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T5 = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T5  = run_swmm_total_flood_ml(mod_inp)
        results_T5.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T5}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T5:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T5 = pd.DataFrame(results_T5)

# Extract numeric gauge number for sensible sorting
df_T5['gauge_num'] = df_T5['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T5 = df_T5.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T5['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T5.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T5 = df_T5.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T5 = vol_matrix_T5.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T5 = 2136.8
percent_decrease_matrix_T5 = (baseline_T5 - vol_matrix_T5) / baseline_T5 * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T5, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 5 Flood Volume (ML) for scaled inflows, baseline: {baseline_T5} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T5, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=5 Percentage Decrease in Flood Volume, baseline {baseline_T5} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_5_wflowforcing.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_10_xsec_bar2.inp")

all_inflow_ts = [
    "Q_2_T_10", "Q_3_T_10", "Q_4_T_10", "Q_5_T_10", "Q_6_T_10", "Q_7_T_10", "Q_8_T_10", "Q_14_T_10", "Q_17_T_10" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T10 = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T10  = run_swmm_total_flood_ml(mod_inp)
        results_T10.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T10}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T10:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T10 = pd.DataFrame(results_T10)

# Extract numeric gauge number for sensible sorting
df_T10['gauge_num'] = df_T10['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T10 = df_T10.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T10['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T10.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T10 = df_T10.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T10 = vol_matrix_T10.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T10 = 2640.1
percent_decrease_matrix_T10 = (baseline_T10 - vol_matrix_T10) / baseline_T10 * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T10, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 10 Flood Volume (ML) for scaled inflows, baseline: {baseline_T10} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T10, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=10 Percentage Decrease in Flood Volume, baseline {baseline_T10} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_10_wflowforcing.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_2_xsec_bar2_culverts.inp")

all_inflow_ts = [
    "Q_2_T_2", "Q_3_T_2", "Q_4_T_2", "Q_5_T_2", "Q_6_T_2", "Q_7_T_2", "Q_8_T_2", "Q_14_T_2", "Q_17_T_2" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T2_cul = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T2_cul  = run_swmm_total_flood_ml(mod_inp)
        results_T2_cul.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T2_cul}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T2_cul:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T2_cul = pd.DataFrame(results_T2_cul)

# Extract numeric gauge number for sensible sorting
df_T2_cul['gauge_num'] = df_T2_cul['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T2_cul = df_T2_cul.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T2_cul['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T2_cul.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T2_cul = df_T2_cul.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T2_cul = vol_matrix_T2_cul.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T2_cul = 1645
percent_decrease_matrix_T2_cul = (baseline_T2_cul - vol_matrix_T2_cul) / baseline_T2_cul * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T2_cul, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 2 (Culverts) Flood Volume (ML) for scaled inflows, baseline: {baseline_T2_cul} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T2_cul, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=2 (Culverts) Percentage Decrease in Flood Volume, baseline {baseline_T2_cul} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_2_wflowforcing_culverts.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_5_xsec_bar2_culverts.inp")

all_inflow_ts = [
    "Q_2_T_5", "Q_3_T_5", "Q_4_T_5", "Q_5_T_5", "Q_6_T_5", "Q_7_T_5", "Q_8_T_5", "Q_14_T_5", "Q_17_T_5" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T5_cul = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T5_cul  = run_swmm_total_flood_ml(mod_inp)
        results_T5_cul.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T5_cul}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T5_cul:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T5_cul = pd.DataFrame(results_T5_cul)

# Extract numeric gauge number for sensible sorting
df_T5_cul['gauge_num'] = df_T5_cul['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T5_cul = df_T5_cul.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T5_cul['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T5_cul.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T5_cul = df_T5_cul.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T5_cul = vol_matrix_T5_cul.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T5_cul = 2425.8
percent_decrease_matrix_T5_cul = (baseline_T5_cul - vol_matrix_T5_cul) / baseline_T5_cul * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T5_cul, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 5 (Culverts) Flood Volume (ML) for scaled inflows, baseline: {baseline_T5_cul} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T5_cul, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=5 (Culverts) Percentage Decrease in Flood Volume, baseline {baseline_T5_cul} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_5_wflowforcing_culverts.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\SWMM_Nakuru_wflow_forcing_reswmm_T_10_xsec_bar2_culverts.inp")

all_inflow_ts = [
    "Q_2_T_10", "Q_3_T_10", "Q_4_T_10", "Q_5_T_10", "Q_6_T_10", "Q_7_T_10", "Q_8_T_10", "Q_14_T_10", "Q_17_T_10" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T10_cul = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T10_cul  = run_swmm_total_flood_ml(mod_inp)
        results_T10_cul.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T10_cul}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T10_cul:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T10_cul = pd.DataFrame(results_T10_cul)

# Extract numeric gauge number for sensible sorting
df_T10_cul['gauge_num'] = df_T10_cul['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T10_cul = df_T10_cul.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T10_cul['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T10_cul.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T10_cul = df_T10_cul.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T10_cul = vol_matrix_T10_cul.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T10_cul = 2944
percent_decrease_matrix_T10_cul = (baseline_T10_cul - vol_matrix_T10_cul) / baseline_T10_cul * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T10_cul, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 5 (Culverts) Flood Volume (ML) for scaled inflows, baseline: {baseline_T10_cul} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T10_cul, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=5 (Culverts) Percentage Decrease in Flood Volume, baseline {baseline_T10_cul} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_10_wflowforcing_culverts.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\2025-05-22 CC\SWMM_Nakuru_wflow_forcing_reswmm_T_2_CC.inp")

all_inflow_ts = [
    "Q_2_T_2", "Q_3_T_2", "Q_4_T_2", "Q_5_T_2", "Q_6_T_2", "Q_7_T_2", "Q_8_T_2", "Q_14_T_2", "Q_17_T_2" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T2_CC = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T2_CC  = run_swmm_total_flood_ml(mod_inp)
        results_T2_CC.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T2_CC}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T2_CC:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T2_CC = pd.DataFrame(results_T2_CC)

# Extract numeric gauge number for sensible sorting
df_T2_CC['gauge_num'] = df_T2_CC['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T2_CC = df_T2_CC.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T2_CC['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T2_CC.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T2_CC = df_T2_CC.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T2_CC = vol_matrix_T2_CC.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T2_CC = 1788.8
percent_decrease_matrix_T2_CC = (baseline_T2_CC - vol_matrix_T2_CC) / baseline_T2_CC * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T2_CC, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 2 (SSP2) Flood Volume (ML) for scaled inflows, baseline: {baseline_T2_CC} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T2_CC, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=2 (SSP2) Percentage Decrease in Flood Volume, baseline {baseline_T2_CC} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_2_wflowforcing_CC.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\2025-05-22 CC\SWMM_Nakuru_wflow_forcing_reswmm_T_5_CC.inp")

all_inflow_ts = [
    "Q_2_T_5", "Q_3_T_5", "Q_4_T_5", "Q_5_T_5", "Q_6_T_5", "Q_7_T_5", "Q_8_T_5", "Q_14_T_5", "Q_17_T_5" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T5_CC = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T5_CC  = run_swmm_total_flood_ml(mod_inp)
        results_T5_CC.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T5_CC}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T5_CC:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T5_CC = pd.DataFrame(results_T5_CC)

# Extract numeric gauge number for sensible sorting
df_T5_CC['gauge_num'] = df_T5_CC['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T5_CC = df_T5_CC.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T5_CC['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T5_CC.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T5_CC = df_T5_CC.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T5_CC = vol_matrix_T5_CC.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T5_CC = 2645.4
percent_decrease_matrix_T5_CC = (baseline_T5_CC - vol_matrix_T5_CC) / baseline_T5_CC * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T5_CC, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 5 (SSP2) Flood Volume (ML) for scaled inflows, baseline: {baseline_T5_CC} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T5_CC, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=5 (SSP2) Percentage Decrease in Flood Volume, baseline {baseline_T5_CC} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_5_wflowforcing_CC.png", dpi=300)

plt.show()

In [ ]:
original_inp = Path(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\2025-05-16 INP files\Return_Periods_5_10_50\2025-05-22 CC\SWMM_Nakuru_wflow_forcing_reswmm_T_10_CC.inp")

all_inflow_ts = [
    "Q_2_T_10", "Q_3_T_10", "Q_4_T_10", "Q_5_T_10", "Q_6_T_10", "Q_7_T_10", "Q_8_T_10", "Q_14_T_10", "Q_17_T_10" # … your 9 inflow time‑series names
    # ...
]

#all_inflow_ts = ["Q_2_T_5"]

scale_factors = [0.8, 0.6, 0.4, 0.2, 0.0 ]

results_T10_CC = []
for ts_name in all_inflow_ts:
    for sf in scale_factors:
        mod_inp = multiply_inflow_mfactor_no_baseline(original_inp, ts_name, sf)
        print(f"\n--- DEBUG for {ts_name}  sf={sf} ---")
        with open(mod_inp, "r") as dbg:
            for L in dbg:
                if ts_name in L and L.strip() and not L.lstrip().startswith(";"):
                    print("Edited line:", L.rstrip())
                    print("Tokens     :", re.split(r"\s+", L.strip()))
                    break
        print("Running:", mod_inp, "\n")

        vol_ml_T10_CC  = run_swmm_total_flood_ml(mod_inp)
        results_T10_CC.append(
            {"timeseries": ts_name,
             "scale_factor": sf,
             "flood_volume_ML": vol_ml_T10_CC}
        )
        print(f"{ts_name}  sf={sf:3}  ➜  {vol_ml_T10_CC:8.2f} ML")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create DataFrame
df_T10_CC = pd.DataFrame(results_T10_CC)

# Extract numeric gauge number for sensible sorting
df_T10_CC['gauge_num'] = df_T10_CC['timeseries'].str.extract(r'Q_(\d+)_').astype(int)

# Sort by scale factor (descending) and gauge number (ascending)
df_T10_CC = df_T10_CC.sort_values(by=['scale_factor', 'gauge_num'], ascending=[False, True])

# Unique sorted lists
scale_factors = sorted(df_T10_CC['scale_factor'].unique(), reverse=True)
gauges_sorted = df_T10_CC.drop_duplicates('timeseries').sort_values('gauge_num')['timeseries'].tolist()

# Pivot to create 2D matrix (Flood Volumes)
vol_matrix_T10_CC = df_T10_CC.pivot(index='scale_factor', columns='timeseries', values='flood_volume_ML')
vol_matrix_T10_CC = vol_matrix_T10_CC.loc[scale_factors, gauges_sorted].values

# Compute percentage decrease relative to baseline 2136.8 ML
baseline_T10_CC = 3194.6
percent_decrease_matrix_T10_CC = (baseline_T10_CC - vol_matrix_T10_CC) / baseline_T10_CC * 100  # in %

# ── Figure & axes ─────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(
    ncols=2,  # two columns, one row
    figsize=(16, 6),
    sharey=True,   # both plots use the same y‑axis labels
    constrained_layout=True  # handles padding automatically
)

clean_gauge_labels = [name.split('_T')[0] for name in gauges_sorted]


# ── Plot 1: Flood Volume ──────────────────────────────────────
im1 = ax1.imshow(vol_matrix_T10_CC, cmap='RdYlGn_r', aspect='auto')
ax1.set_xticks(np.arange(len(gauges_sorted)))
ax1.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax1.set_yticks(np.arange(len(scale_factors)))
ax1.set_yticklabels(scale_factors)
ax1.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale Factor', fontsize=12)
ax1.set_title(f'T= 10 (SSP2) Flood Volume (ML) for scaled inflows, baseline: {baseline_T10_CC} ML', fontsize=14)
#fig.colorbar(im1, ax=ax1, label='Flood Volume (ML)')
cbar1 = fig.colorbar(im1, ax=ax1)
cbar1.set_label('Flood Volume (ML)', fontsize=12)
cbar1.ax.tick_params(labelsize=12)

# ── Plot 2: Percentage Decrease ───────────────────────────────
im2 = ax2.imshow(percent_decrease_matrix_T10_CC, cmap='RdYlGn', aspect='auto')
ax2.set_xticks(np.arange(len(gauges_sorted)))
ax2.set_xticklabels(clean_gauge_labels, rotation=45, ha='right', fontsize=12)
ax2.set_xlabel('Gauge', fontsize=12)
ax1.set_ylabel('Scale factor', fontsize=12)
ax2.set_title(f'T=10 (SSP2) Percentage Decrease in Flood Volume, baseline {baseline_T10_CC} ML', fontsize=14)
#fig.colorbar(im2, ax=ax2, label='% Decrease from baseline')
cbar2 = fig.colorbar(im2, ax=ax2)
cbar2.set_label('% Decrease from baseline', fontsize=12)
cbar2.ax.tick_params(labelsize=12)

# ── Show figure ───────────────────────────────────────────────

plt.savefig("Figures/imshow_flood_volume_analysisT_10_wflowforcing_CC.png", dpi=300)

plt.show()

In [ ]:
# Suppose you have a list of result DataFrames, each with columns 'scale_factor' and 'percentage'
percent_decrease_lists = [percent_decrease_matrix_T2, percent_decrease_matrix_T5, percent_decrease_matrix_T10, percent_decrease_matrix_T2_CC, percent_decrease_matrix_T5_CC, percent_decrease_matrix_T10_CC, percent_decrease_matrix_T2_cul, percent_decrease_matrix_T5_cul, percent_decrease_matrix_T10_cul]


# #percent_decrease_dfs.to_csv("percent_decrease_dfs.csv", index=False)
# # Sum percentages per scale factor for each result DataFrame
# summed_dfs = []
# for i, list in enumerate(percent_decrease_lists):
#     df = pd.DataFrame(list)
#     display(df)
#     df.to_csv(f"percent_decrease_matrix_T{i+1}.csv", index=False)
#     # summed = df.groupby('scale_factor')['percentage'].sum().reset_index()
#     # summed['scenario'] = f"Scenario_{i+1}"  # or use your actual scenario names
#     # summed_dfs.append(summed)

# # Combine all into a single DataFrame
# all_summed = pd.concat(summed_dfs, ignore_index=True)

# print(all_summed)


# List of scenario names for file naming and scenario column
scenario_names = [
    "T2", "T5", "T10",
    "T2_CC", "T5_CC", "T10_CC",
    "T2_cul", "T5_cul", "T10_cul"
]

# If you want to group by the index (assuming the index is meaningful, e.g., scale_factor)
# and sum the percentages for each DataFrame:

# Define your gauge names in the correct order
gauge_names = ['Q_2', 'Q_3', 'Q_4', 'Q_5', 'Q_6', 'Q_7', 'Q_8', 'Q_14', 'Q_17']

# If you want to group by the index (assuming the index is meaningful, e.g., scale_factor)
# and sum the percentages for each DataFrame:

summed_dfs = []
for i, matrix in enumerate(percent_decrease_lists):
    df = pd.DataFrame(matrix)
    df.columns = gauge_names
    #df_melted = df.melt(var_name='gauge', value_name='percentage')
    # Set the index if not already set (e.g., to 'scale_factor')
    if 'scale_factor' in df.columns:
        df = df.set_index('scale_factor')
    # Group by the index and sum percentages
    summed = df.groupby(df.index)[gauge_names].sum().reset_index()
    summed['scenario'] = scenario_names[i]
    summed_dfs.append(summed)
    # Save each summed DataFrame to a separate CSV
    summed.to_csv(f"percent_decrease_{scenario_names[i]}.csv", index=False)
# Combine all into a single DataFrame
all_summed = pd.concat(summed_dfs, ignore_index=True)

print(all_summed)